This notebooks is based on the script [run_summarization_no_trainer.py](https://github.com/huggingface/transformers/blob/v4.12.5/examples/pytorch/summarization/run_summarization_no_trainer.py) from HuggingFace.

In [1]:
import datasets
import nltk
import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from torch.utils.data import DataLoader

In [2]:
tokenizer = AutoTokenizer.from_pretrained('t5-small', use_fast=True,
                                          cache_dir='./cache/t5-small_tokenizer')

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    't5-small',
    cache_dir='./cache/t5-small_model'
)

In [4]:
raw_datasets = load_dataset('xsum')

Using custom data configuration default
Reusing dataset xsum (/users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
max_source_length = 1024
max_target_length = 128
ignore_pad_token_for_loss = True
padding = False            # else 'max_length'
label_pad_token_id = -100  # else tokenizer.pad_token_id
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

In [6]:
def preprocess_function(examples, text_column='document',
                        summary_column='summary', prefix='summarize: '):
    inputs = examples[text_column]
    targets = examples[summary_column]
    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=max_source_length,
                             padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length,
                           padding=padding, truncation=True)

    # padding in the loss.
    if padding == "max_length" and ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100)
             for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id
)

In [8]:
processed_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    # load_from_cache_file=not args.overwrite_cache,
    desc="Running tokenizer on dataset",
)

Loading cached processed dataset at /users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-a467f0bf6a81dfb9.arrow
Loading cached processed dataset at /users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-0363403826d0851f.arrow
Loading cached processed dataset at /users/sarafael/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-4c41ee4419d04a63.arrow


In [9]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

In [10]:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=data_collator, batch_size=per_device_train_batch_size
)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=data_collator,
    batch_size=per_device_eval_batch_size
)

In [11]:
weight_decay = 0.0
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters()
                   if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters()
                   if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

In [12]:
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)

In [13]:
device = 0
model.to(device)
model.train();

In [14]:
for step, batch in enumerate(train_dataloader):
    if step > 5:
        break
    
    optimizer.zero_grad()
    outputs = model(**batch.to(device))
    loss = outputs.loss
    loss.backward()
    optimizer.step()

## Evaluation

In [15]:
metric = load_metric("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [16]:
model.eval()

gen_kwargs = {
    "max_length": 128,   # args.val_max_target_length if args is not None else config.max_length,
    "num_beams": None,   # args.num_beams,
}
for step, batch in enumerate(eval_dataloader):
    if step > 20:
        break

    with torch.no_grad():
        generated_tokens = model.generate(
            batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            # **gen_kwargs,
        )

        labels = batch["labels"]
        generated_tokens = generated_tokens.cpu().numpy()
        
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

result = metric.compute(use_stemmer=True)
# Extract a few results from ROUGE
result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

result = {k: round(v, 4) for k, v in result.items()}

print(result)

{'rouge1': 15.3906, 'rouge2': 1.0662, 'rougeL': 12.1903, 'rougeLsum': 12.5174}


In [ ]:
for epoch in range(1):
    for i, batch in tqdm(enumerate(train_loader)):
        optim.zero_grad()
        outputs = model(input_ids=batch[0].to(device),
              token_type_ids=batch[1].to(device),
              attention_mask=batch[2].to(device),
              start_positions=batch[3].to(device),
              end_positions=batch[4].to(device)
             )
        
        loss = outputs[0]
        loss.backward()
        optim.step()
        
        #if i > 10:
        #    break

In [ ]:
for epoch in range(args.num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps += 1